In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, time
from google.cloud import bigquery
from google.oauth2 import service_account
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
# pd.reset_option(‘max_columns’)

In [2]:
path_to_private_key = './ieso-dashboard-c639f1a39298.json'
credentials = service_account.Credentials.from_service_account_file(
    path_to_private_key,
    scopes=['https://www.googleapis.com/auth/cloud-platform'],
)

### Independent/ Input Variables 

In [3]:
# OLD DATA from csv file
old_ade=pd.read_csv('adequacy_202209052348.csv')
old_ade['mkt_datetime']=old_ade['mkt_datetime'].map(lambda x: datetime.strptime(x,'%m/%d/%Y %H:%M').isoformat(timespec='minutes'))

In [4]:
# CURRENT DATA from bigQuery
query_string="SELECT * FROM `ieso-dashboard.Adequacy2.Adequacy2_table` ORDER BY mkt_date, mkt_he;"
gbq_ade = pd.read_gbq(query_string, credentials=credentials)

In [5]:
hour_col = gbq_ade['mkt_he'].apply(lambda x: str(x-1)+":00")
date_col = gbq_ade['mkt_date'].apply(lambda x: str(x))

date_time=pd.to_datetime(date_col +' '+ hour_col).map(lambda x: x.isoformat(timespec='minutes'))

gbq_ade.insert(loc = 0,
          column = 'mkt_datetime',
          value = date_time)
# gbq_ade['mkt_datetime']=date_time

In [6]:
ind_var = ['mkt_datetime','int_nuc_cap','int_nuc_out','int_nuc_off','int_nuc_sch','int_gas_cap','int_gas_out','int_gas_off','int_gas_sch','int_hyd_cap','int_hyd_out','int_hyd_for','int_hyd_off','int_hyd_sch','int_win_cap','int_win_out','int_win_for','int_win_sch','int_sol_cap','int_sol_out','int_sol_for','int_sol_sch','int_bio_cap','int_bio_out','int_bio_off','int_bio_sch','int_oth_cap','int_oth_out','int_oth_off','int_oth_sch','int_tot_out','int_tot_off','int_tot_sch','imp_man_off','imp_man_sch','imp_min_off','imp_min_sch','imp_mic_off','imp_mic_sch','imp_new_off','imp_new_sch','imp_que_off','imp_que_sch','imp_tot_off','imp_tot_sch','imp_tot_est','imp_tot_cap','ont_bot_cap','ont_reg','ont_tot_sup','ont_for_dem','ont_peak_dem','ont_avg_dem','ont_emb_wind','ont_emb_sol','ont_disp_cap','ont_disp_bid','ont_disp_on','ont_disp_off','ont_hdr_bid','ont_hdr_sch','ont_hdr_cur','exp_man_bid','exp_man_sch','exp_min_bid','exp_min_sch','exp_mic_bid','exp_mic_sch','exp_new_bid','exp_new_sch','exp_que_bid','exp_que_sch','exp_tot_bid','exp_tot_sch','exp_tot_cap','grh_tot','grh_min_10n','grh_min_10s','grh_lfu','grh_add','total_req','cap_excess','energy_excess','offered_cap_excess','res_not_sch','imp_not_sch']
X_curr = gbq_ade[ind_var]
X_old = old_ade[ind_var]

In [7]:
X1=X_curr[X_curr['mkt_datetime']>max(X_old['mkt_datetime'])]  # which is '2022-09-06T23:00'
X_combined = pd.concat([X_old, X1], ignore_index=True)

In [8]:
X_combined.loc[:,ind_var] = X_combined.loc[:,ind_var].ffill()
X=X_combined.dropna() # includes all days from 1/1/20 till current date

### Target Variable (ont_ene)

In [9]:
## CURRENT DATA being fetched from bigQuery
query_string="SELECT * FROM `ieso-dashboard.RealtimeMktPrice.RealtimeMktPrice_table` ORDER BY mkt_date, mkt_he;"
gbq_rtmp = pd.read_gbq(query_string, credentials=credentials)

In [10]:
hour_col = gbq_rtmp['mkt_he'].apply(lambda x: str(x-1)+":00")
date_col = gbq_rtmp['mkt_date'].apply(lambda x: str(x))

date_time=pd.to_datetime(date_col +' '+ hour_col).map(lambda x: x.isoformat(timespec='minutes'))

gbq_rtmp.insert(loc = 0,
          column = 'mkt_datetime',
          value = date_time)

In [11]:
dep_var = ['mkt_datetime','ont_ene']
Y_curr = gbq_rtmp[dep_var]

In [12]:
## OLD DATA
old_rtmp=pd.read_excel('historical ont_ene.xlsx',skiprows=4)
hour_col = old_rtmp['he'].apply(lambda x: str(x-1)+":00")
date_col = old_rtmp['mkt_date'].apply(lambda x: str(x))

date_time=pd.to_datetime(date_col +' '+ hour_col).map(lambda x: x.isoformat(timespec='minutes'))
old_rtmp.insert(loc = 0,
          column = 'mkt_datetime',
          value = date_time)

In [13]:
val_to_fillna=old_rtmp[old_rtmp['mkt_date']=='2022-05-20'].ont_ene.mean()
old_rtmp.at[20893,'ont_ene']=val_to_fillna
old_rtmp.at[20894,'ont_ene']=val_to_fillna
old_rtmp.at[20895,'ont_ene']=val_to_fillna

# old_rtmpold_rtmp['mkt_datetime']=='2022-05-20T14:00']['ont_ene']=val_to_fillna
# old_rtmp[old_rtmp['mkt_datetime']=='2022-05-20T15:00']['ont_ene']=val_to_fillna
old_rtmp=old_rtmp.dropna()

In [14]:
Y_old = old_rtmp[dep_var]
Y1=Y_curr[Y_curr['mkt_datetime']>max(Y_old['mkt_datetime'])]  # which is '2022-09-07T10:00'
Y_combined = pd.concat([Y_old, Y1], ignore_index=True)

In [15]:
Y=Y_combined.dropna() # includes all days from 1/1/20 till current date

### Merging all the columns

In [16]:
X[X['mkt_datetime'].duplicated() == True]

,mkt_datetime,int_nuc_cap,int_nuc_out,int_nuc_off,int_nuc_sch,int_gas_cap,int_gas_out,int_gas_off,int_gas_sch,int_hyd_cap,int_hyd_out,int_hyd_for,int_hyd_off,int_hyd_sch,int_win_cap,int_win_out,int_win_for,int_win_sch,int_sol_cap,int_sol_out,int_sol_for,int_sol_sch,int_bio_cap,int_bio_out,int_bio_off,int_bio_sch,int_oth_cap,int_oth_out,int_oth_off,int_oth_sch,int_tot_out,int_tot_off,int_tot_sch,imp_man_off,imp_man_sch,imp_min_off,imp_min_sch,imp_mic_off,imp_mic_sch,imp_new_off,imp_new_sch,imp_que_off,imp_que_sch,imp_tot_off,imp_tot_sch,imp_tot_est,imp_tot_cap,ont_bot_cap,ont_reg,ont_tot_sup,ont_for_dem,ont_peak_dem,ont_avg_dem,ont_emb_wind,ont_emb_sol,ont_disp_cap,ont_disp_bid,ont_disp_on,ont_disp_off,ont_hdr_bid,ont_hdr_sch,ont_hdr_cur,exp_man_bid,exp_man_sch,exp_min_bid,exp_min_sch,exp_mic_bid,exp_mic_sch,exp_new_bid,exp_new_sch,exp_que_bid,exp_que_sch,exp_tot_bid,exp_tot_sch,exp_tot_cap,grh_tot,grh_min_10n,grh_min_10s,grh_lfu,grh_add,total_req,cap_excess,energy_excess,offered_cap_excess,res_not_sch,imp_not_sch
1611,2020-03-08T03:00,13032,3030,10115,10020,9829,1379,4072,178,7824,971,4143,5777,3189,4483,1618,2865,2842,424,424,0,0,480,261,4,4,4,0,0,0,8219,22834,16233,355,94,160,0,409,0,4,0,994,9,1922,103,0,0,1449,100,26303,12989,13056,12989,196,0,481,276,-273,3,0.0,0.0,0.0,289,-211,193,-49,2332,-1450,1653,-1407,1776,-432,6243,-3549,629,1418,945,236,0,0,15033.0,11270.0,11140,10558.0,6874.0,1819
10515,2021-03-14T03:00,13081,4768,9554,8345,10833,1108,3316,216,7415,973,3061,5507,2768,4783,2373,2410,2830,478,478,0,0,480,286,4,4,0,0,7,0,10507,20798,14163,275,96,305,69,420,0,80,0,2077,9,3157,174,0,0,939,100,25523,12793,12897,12793,111,0,467,255,-248,7,0.0,0.0,0.0,444,-86,233,-79,1545,-1000,1072,-517,1090,-66,4384,-1748,500,1418,945,236,0,0,14704.0,10819.0,9870,9895.0,6883.0,2983
19251,2022-03-13T03:00,13081,4749,9247,8345,10799,2033,5078,827,7740,1076,2965,6059,3461,4843,2552,2291,2274,478,478,0,0,315,133,2,2,0,0,0,0,11499,22676,14909,142,69,165,0,280,0,0,0,1954,1549,2541,1618,0,0,2285,36,23435,14898,14974,14898,177,0,475,281,-246,35,0.0,0.0,0.0,76,-12,84,-20,1802,-1450,443,-168,1423,-214,3828,-1864,500,1418,945,236,0,0,16781.0,6654.0,5316,9106.0,8014.0,923


In [17]:
X.at[1610,'mkt_datetime']='2020-03-08T02:00'
X.at[10514,'mkt_datetime']='2021-03-14T02:00'
X.at[19250,'mkt_datetime']='2022-03-13T02:00'

In [18]:
X['mkt_datetime'] = pd.to_datetime(X.mkt_datetime)
Y['mkt_datetime'] = pd.to_datetime(Y.mkt_datetime)

merged_data = pd.merge(X,Y,how='left',on = 'mkt_datetime')
merged_data=merged_data.set_index('mkt_datetime')

In [19]:
merged_data.to_csv('data.csv')